## **Federated Learning for attack detection: 7 nodes sharing gradients: FedAVG**

IDs from this file = **id8xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [5]:
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_18580/3836997398.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [6]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

### Experiments with datasets

#### Dataset 7A

In [7]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part5.csv', low_memory=False)
training6 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part6.csv', low_memory=False)
training7 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part7.csv', low_memory=False)

In [8]:
# Define 
num_nodes = 7
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [9]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id800.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [10]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [11]:
global_model = build_model((24,1,1))

In [13]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
x6, y6 = preprocessing(training6)
x7, y7 = preprocessing(training7)

In [14]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [15]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        elif node == 4: 
            x, y = x5, y5
        elif node == 5:
            x, y = x6, y6
        elif node == 6:
            x, y = x7, y7
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id800.hdf5')

Epoch 1/5
22/22 [==============================] - 4s 180ms/step - loss: 0.4786 - accuracy: 0.9559 - val_loss: 1.1013 - val_accuracy: 0.6185
Epoch 2/5
22/22 [==============================] - 4s 167ms/step - loss: 0.1121 - accuracy: 0.9893 - val_loss: 2.1094 - val_accuracy: 0.6187
Epoch 3/5
22/22 [==============================] - 4s 165ms/step - loss: 0.0499 - accuracy: 0.9915 - val_loss: 0.9106 - val_accuracy: 0.6191
Epoch 4/5
22/22 [==============================] - 4s 168ms/step - loss: 0.0309 - accuracy: 0.9918 - val_loss: 0.5823 - val_accuracy: 0.6495
Epoch 5/5
22/22 [==============================] - 4s 175ms/step - loss: 0.0279 - accuracy: 0.9921 - val_loss: 0.6174 - val_accuracy: 0.6454
Epoch 1/5
22/22 [==============================] - 4s 175ms/step - loss: 0.0270 - accuracy: 0.9917 - val_loss: 0.5414 - val_accuracy: 0.6688
Epoch 2/5
22/22 [==============================] - 4s 178ms/step - loss: 0.0244 - accuracy: 0.9919 - val_loss: 0.3880 - val_accuracy: 0.7462
Epoch 3/5
22/

In [16]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.05185585841536522, 0.22198174893856049], [0.07023578882217407, 0.4187829792499542], [0.0698380097746849, 0.3980315923690796]]
Accuracy for iterations:  [[0.9740716814994812, 0.9038363695144653], [0.9720485806465149, 0.8064193725585938], [0.9754082560539246, 0.81581050157547]]
F1 for iterations:  [[0.9740397749331544, 0.9022309916842483], [0.9720038590077885, 0.794770736312494], [0.975379422934498, 0.8056475489028748]]
Precision for iterations:  [[0.9745438223826113, 0.9129364335975925], [0.9727745762291028, 0.8476057364886238], [0.9758466743376849, 0.8536176953832524]]
Recall for iterations:  [[0.974071693593152, 0.9038363728838189], [0.9720485553185895, 0.8064193853084357], [0.9754082795290543, 0.8158105064302841]]


#### Dataset 7B

In [17]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part5.csv', low_memory=False)
training6 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part6.csv', low_memory=False)
training7 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part7.csv', low_memory=False)

In [18]:
# Define 
num_nodes = 7
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [19]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id801.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [20]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [21]:
global_model = build_model((24,1,1))

In [22]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
x6, y6 = preprocessing(training6)
x7, y7 = preprocessing(training7)

In [23]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [24]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        elif node == 5:
            x, y = x6, y6
        elif node == 6:
            x, y = x7, y7
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id801.hdf5')

Epoch 1/5
22/22 [==============================] - 4s 180ms/step - loss: 0.4743 - accuracy: 0.9491 - val_loss: 0.4989 - val_accuracy: 0.8740
Epoch 2/5
22/22 [==============================] - 4s 180ms/step - loss: 0.1137 - accuracy: 0.9793 - val_loss: 0.6969 - val_accuracy: 0.8742
Epoch 3/5
22/22 [==============================] - 4s 174ms/step - loss: 0.0515 - accuracy: 0.9914 - val_loss: 0.2355 - val_accuracy: 0.8751
Epoch 4/5
22/22 [==============================] - 4s 195ms/step - loss: 0.0305 - accuracy: 0.9914 - val_loss: 0.1829 - val_accuracy: 0.8836
Epoch 5/5
22/22 [==============================] - 5s 205ms/step - loss: 0.0264 - accuracy: 0.9920 - val_loss: 0.1898 - val_accuracy: 0.8813
Epoch 1/5
22/22 [==============================] - 4s 202ms/step - loss: 0.0252 - accuracy: 0.9909 - val_loss: 0.3168 - val_accuracy: 0.8259
Epoch 2/5
22/22 [==============================] - 4s 197ms/step - loss: 0.0225 - accuracy: 0.9912 - val_loss: 0.4866 - val_accuracy: 0.6810
Epoch 3/5
22/

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1721/1721 [==============================] - 5s 3ms/step - loss: 6.3833 - accuracy: 0.5608


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
22/22 [==============================] - 4s 174ms/step - loss: 0.8329 - accuracy: 0.8259 - val_loss: 0.0394 - val_accuracy: 0.9966
Epoch 2/5
22/22 [==============================] - 4s 162ms/step - loss: 0.0265 - accuracy: 0.9916 - val_loss: 0.1742 - val_accuracy: 0.9227
Epoch 3/5
22/22 [==============================] - 4s 168ms/step - loss: 0.0212 - accuracy: 0.9937 - val_loss: 0.1303 - val_accuracy: 0.9323
Epoch 4/5
22/22 [==============================] - 4s 173ms/step - loss: 0.0192 - accuracy: 0.9935 - val_loss: 0.1402 - val_accuracy: 0.9122
Epoch 5/5
22/22 [==============================] - 4s 182ms/step - loss: 0.0181 - accuracy: 0.9945 - val_loss: 0.1216 - val_accuracy: 0.9188
Epoch 1/5
22/22 [==============================] - 4s 189ms/step - loss: 0.0170 - accuracy: 0.9943 - val_loss: 0.2280 - val_accuracy: 0.8723
Epoch 2/5
22/22 [==============================] - 4s 177ms/step - loss: 0.0148 - accuracy: 0.9948 - val_loss: 0.2571 - val_accuracy: 0.8537
Epoch 3/5
22/

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
22/22 [==============================] - 7s 321ms/step - loss: 0.4322 - accuracy: 0.8757 - val_loss: 0.0492 - val_accuracy: 0.9888
Epoch 2/5
22/22 [==============================] - 4s 190ms/step - loss: 0.0193 - accuracy: 0.9933 - val_loss: 0.1926 - val_accuracy: 0.8968
Epoch 3/5
22/22 [==============================] - 5s 222ms/step - loss: 0.0169 - accuracy: 0.9949 - val_loss: 0.1429 - val_accuracy: 0.9072
Epoch 4/5
22/22 [==============================] - 5s 245ms/step - loss: 0.0157 - accuracy: 0.9954 - val_loss: 0.1771 - val_accuracy: 0.8980
Epoch 5/5
22/22 [==============================] - 5s 232ms/step - loss: 0.0150 - accuracy: 0.9958 - val_loss: 0.1849 - val_accuracy: 0.8983
Epoch 1/5
22/22 [==============================] - 6s 284ms/step - loss: 0.0141 - accuracy: 0.9954 - val_loss: 0.2551 - val_accuracy: 0.8687
Epoch 2/5
22/22 [==============================] - 6s 252ms/step - loss: 0.0139 - accuracy: 0.9954 - val_loss: 0.1720 - val_accuracy: 0.9168
Epoch 3/5
22/

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1721/1721 [==============================] - 10s 6ms/step - loss: 4.4292 - accuracy: 0.5608


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[5.471231460571289, 6.383303642272949], [4.70941162109375, 5.026413440704346], [4.249748706817627, 4.4292378425598145]]
Accuracy for iterations:  [[0.5287913680076599, 0.5607607364654541], [0.5287913680076599, 0.5607607364654541], [0.5287913680076599, 0.5607607364654541]]
F1 for iterations:  [[0.36580572534487127, 0.4029478640083462], [0.36580572534487127, 0.4029478640083462], [0.36580572534487127, 0.4029478640083462]]
Precision for iterations:  [[0.2796203190506846, 0.31445260225958305], [0.2796203190506846, 0.31445260225958305], [0.2796203190506846, 0.31445260225958305]]
Recall for iterations:  [[0.5287913757340267, 0.5607607353048027], [0.5287913757340267, 0.5607607353048027], [0.5287913757340267, 0.5607607353048027]]


#### Dataset 7C

In [26]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part5.csv', low_memory=False)
training6 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part6.csv', low_memory=False)
training7 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part7.csv', low_memory=False)

In [27]:
# Define 
num_nodes = 7
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [28]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id802.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [29]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [30]:
global_model = build_model((24,1,1))

In [31]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
x6, y6 = preprocessing(training6)
x7, y7 = preprocessing(training7)

In [32]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [33]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        elif node == 4: 
            x, y = x5, y5
        elif node == 5:
            x, y = x6, y6
        elif node == 6:
            x, y = x7, y7
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id802.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 197ms/step - loss: 0.5271 - accuracy: 0.9335 - val_loss: 0.5175 - val_accuracy: 0.8267
Epoch 2/5
16/16 [==============================] - 3s 198ms/step - loss: 0.1387 - accuracy: 0.9837 - val_loss: 0.3878 - val_accuracy: 0.8273
Epoch 3/5
16/16 [==============================] - 3s 174ms/step - loss: 0.0488 - accuracy: 0.9881 - val_loss: 0.1423 - val_accuracy: 0.9334
Epoch 4/5
16/16 [==============================] - 3s 175ms/step - loss: 0.0355 - accuracy: 0.9892 - val_loss: 0.1701 - val_accuracy: 0.9141
Epoch 5/5
16/16 [==============================] - 3s 171ms/step - loss: 0.0300 - accuracy: 0.9894 - val_loss: 0.1579 - val_accuracy: 0.9194
Epoch 1/5
32/32 [==============================] - 7s 204ms/step - loss: 0.1104 - accuracy: 0.9695 - val_loss: 0.8676 - val_accuracy: 0.7491
Epoch 2/5
32/32 [==============================] - 6s 192ms/step - loss: 0.0313 - accuracy: 0.9939 - val_loss: 0.8984 - val_accuracy: 0.7480
Epoch 3/5
32/

In [34]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.048929013311862946, 0.05968993902206421], [0.038720790296792984, 0.0770510658621788], [0.09274330735206604, 0.30963996052742004]]
Accuracy for iterations:  [[0.9910821318626404, 0.9902274012565613], [0.9916810393333435, 0.9880840182304382], [0.9608519077301025, 0.831868052482605]]
F1 for iterations:  [[0.9910852850645745, 0.9902372314709055], [0.9916839000686489, 0.9880921646613824], [0.9607299641024801, 0.8239361419268945]]
Precision for iterations:  [[0.9911854671535346, 0.9904017524130945], [0.9917778611019462, 0.9881687462082935], [0.9628368207667298, 0.8640875336071753]]
Recall for iterations:  [[0.991082123345701, 0.9902274213470901], [0.9916810307049578, 0.9880839933154109], [0.9608519091998013, 0.8318680520235414]]
